# Historical Price Explorer

Interactively fetch dividend/split-adjusted prices from Yahoo Finance, then visualize the closing trend. Use the controls below to pick a ticker and date range before pressing *Fetch data*.

In [1]:
from __future__ import annotations

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

plt.style.use("seaborn-v0_8-whitegrid")
pd.options.display.float_format = '{:.2f}'.format


In [2]:
default_end = pd.Timestamp.today().normalize()
default_start = default_end - pd.DateOffset(years=10)

ticker_input = widgets.Text(
    value="AAPL",
    description="Ticker",
    placeholder="e.g. AAPL",
    disabled=False,
)
start_input = widgets.DatePicker(
    description="Start",
    value=default_start.to_pydatetime(),
)
end_input = widgets.DatePicker(
    description="End",
    value=default_end.to_pydatetime(),
)
run_button = widgets.Button(description="Fetch data", button_style="primary")
output = widgets.Output()

def _to_timestamp(value):
    if value is None:
        return None
    return pd.Timestamp(value)

def fetch_and_plot(_=None):
    with output:
        output.clear_output()
        ticker = ticker_input.value.strip().upper()
        start = _to_timestamp(start_input.value)
        end = _to_timestamp(end_input.value)
        if not ticker:
            print("Please enter a ticker symbol.")
            return
        if start is None or end is None:
            print("Please select both start and end dates.")
            return
        if start >= end:
            print("Start date must be before end date.")
            return
        try:
            history = yf.Ticker(ticker).history(
                start=start,
                end=end + pd.Timedelta(days=1),
                auto_adjust=True,
            )
        except Exception as exc:  # pragma: no cover
            print(f"Failed to download data: {exc}")
            return
        if history.empty:
            print("No data returned for the selected period.")
            return
        adj_close = history["Close"].rename("Adj Close")
        high_date = adj_close.idxmax()
        low_date = adj_close.idxmin()
        high_price = adj_close.loc[high_date]
        low_price = adj_close.loc[low_date]
        display(adj_close.to_frame().head())
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(adj_close.index, adj_close.values, label=f"{ticker} Adj Close")
        ax.scatter(
            high_date,
            high_price,
            color="tab:red",
            marker="^",
            s=80,
            label=f"High {high_price:.2f} on {high_date.date()}",
        )
        ax.scatter(
            low_date,
            low_price,
            color="tab:green",
            marker="v",
            s=80,
            label=f"Low {low_price:.2f} on {low_date.date()}",
        )
        ax.annotate(
            f"High {high_price:.2f}\n{high_date:%Y-%m-%d}",
            xy=(high_date, high_price),
            xytext=(8, 8),
            textcoords="offset points",
            color="tab:red",
            fontsize=9,
        )
        ax.annotate(
            f"Low {low_price:.2f}\n{low_date:%Y-%m-%d}",
            xy=(low_date, low_price),
            xytext=(8, -14),
            textcoords="offset points",
            color="tab:green",
            fontsize=9,
        )
        ax.set_title(f"{ticker} adjusted close price")
        ax.set_ylabel("Price (USD)")
        ax.set_xlabel("Date")
        ax.legend()
        fig.autofmt_xdate()
        plt.show()

run_button.on_click(fetch_and_plot)

controls = widgets.VBox([ticker_input, widgets.HBox([start_input, end_input]), run_button])
display(controls, output)

Output()